In [1]:
pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("WELFake_Dataset.csv")

In [4]:
df.head(5)

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [5]:
df.describe()

,Unnamed: 0,label
count,72134.000000,72134.000000
mean,36066.500000,0.514404
std,20823.436496,0.499796
min,0.000000,0.000000
25%,18033.250000,0.000000
50%,36066.500000,1.000000
75%,54099.750000,1.000000
max,72133.000000,1.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72134 entries, 0 to 72133
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  72134 non-null  int64 
 1   title       71576 non-null  object
 2   text        72095 non-null  object
 3   label       72134 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 2.2+ MB


In [7]:
df.isnull().sum()

Unnamed: 0      0
title         558
text           39
label           0
dtype: int64

In [8]:
df=df.fillna('')

In [9]:
df.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [10]:
df.columns

Index(['Unnamed: 0', 'title', 'text', 'label'], dtype='object')

In [11]:
df=df.drop(['Unnamed: 0','title'],axis=1)

In [12]:
df

,text,label
0,No comment is expected from Barack Obama Membe...,1
1,Did they post their votes for Hillary already?,1
2,"Now, most of the demonstrators gathered last ...",1
3,A dozen politically active pastors came here f...,0
4,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
...,...,...
72129,WASHINGTON (Reuters) - Hackers believed to be ...,0
72130,"You know, because in fantasyland Republicans n...",1
72131,Migrants Refuse To Leave Train At Refugee Camp...,0
72132,MEXICO CITY (Reuters) - Donald Trump’s combati...,0


In [13]:
df.head()

,text,label
0,No comment is expected from Barack Obama Membe...,1
1,Did they post their votes for Hillary already?,1
2,"Now, most of the demonstrators gathered last ...",1
3,A dozen politically active pastors came here f...,0
4,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [14]:
import nltk

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rawat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
 from nltk.corpus import stopwords

In [17]:
from nltk.stem.porter import PorterStemmer

In [18]:
import re

In [19]:
port_stem=PorterStemmer()

In [20]:
port_stem

<PorterStemmer>

In [21]:
def stemming(content):
    con =re.sub('[^a-zA-Z]', ' ',content)
    con=con.lower()
    con=con.split()
    con=[port_stem.stem(word) for word in con if not word in stopwords.words('english')]
    con=' '.join(con)
    return con
    

In [22]:
stemming('hi there  is a book')

'hi book'

In [23]:

from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def stemming(text):
    return ' '.join([stemmer.stem(word) for word in text.split()])

def fast_stemming(texts):
    return [stemming(t) for t in texts]

batch_size = 5000
processed_texts = []

for start in range(0, len(df), batch_size):
    end = start + batch_size
    batch = df['text'].iloc[start:end]
    print(f"Processing rows {start} to {end}...")
    processed_texts.extend(fast_stemming(batch))

df['text'] = processed_texts
print("✅ Stemming complete!")

Processing rows 0 to 5000...
Processing rows 5000 to 10000...
Processing rows 10000 to 15000...
Processing rows 15000 to 20000...
Processing rows 20000 to 25000...
Processing rows 25000 to 30000...
Processing rows 30000 to 35000...
Processing rows 35000 to 40000...
Processing rows 40000 to 45000...
Processing rows 45000 to 50000...
Processing rows 50000 to 55000...
Processing rows 55000 to 60000...
Processing rows 60000 to 65000...
Processing rows 65000 to 70000...
Processing rows 70000 to 75000...
✅ Stemming complete!


In [24]:
x=df['text']

In [25]:
y=df['label']

In [47]:
x.shape

(72134,)

In [48]:
from sklearn.model_selection import train_test_split

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [50]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)

In [62]:
vect=TfidfVectorizer(max_features=3000,stop_words='english')

In [63]:
print(type(x_train[0]))

<class 'str'>


In [64]:
x_train_vect=vect.fit_transform(x_train)
x_test_vect=vect.transform(x_test)

In [65]:
x_train_vect.shape

(57707, 3000)

In [66]:
from sklearn.tree import DecisionTreeClassifier

In [72]:
model=DecisionTreeClassifier(max_depth=10,min_samples_split=10,max_features=0.3,random_state=42)

In [73]:
model.fit(x_train_vect,y_train)

DecisionTreeClassifier(max_depth=10, max_features=0.3, min_samples_split=10,
                       random_state=42)

In [75]:
prediction=model.predict(x_test_vect)

In [76]:
prediction

array([1, 1, 0, ..., 1, 1, 1])

In [79]:
model.score(x_test_vect,y_test)

0.8922159839190407

In [80]:
import pickle

In [81]:
pickle.dump(vect,open('vector.pkl','wb'))

In [82]:
pickle.dump(model,open('model.pkl','wb'))

In [84]:
vector_form=pickle.load(open('vector.pkl','rb'))

In [85]:
load_model=pickle.load(open('model.pkl','rb'))

In [86]:
def fake_news(news):
    news=stemming(news)
    input_data=[news]
    vector_form1=vector_form.transform(input_data)
    prediction=load_model.predict(vector_form1)
    return prediction

In [88]:
val=fake_news(""" Now, most of the demonstrators gathered last night were exercising their constitutional and protected right to peaceful protest in order to raise issues and create change.    Loretta Lynch aka Eric Holder in a skirt""")

In [90]:
if val==[0]:
    print('reliable')
else:
    print('unreliable')

unreliable
